Unfortunately Github doesn't render ipywidgets, which allow us to interactively alter the parameters of the model. If you want to mess with the parameters yourself then download this file, and also install and activate ipywidgets, which you can do in the command line interface with the following commands:
```
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

In [1]:
#dependencies
import numpy as np
import operator as op
from functools import reduce
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

Parameters:
<br>
$\lambda$ = probability of encountering resources <br>
$S(t)$ = number of solitary locusts at time t <br>
$G(t)$ = number of gregarious locusts at time t <br>
$R(t)$ = number of resources at time t <br>
$\alpha$ = percentage of total resources found in a patch, proxy for clumpiness <br>
$k$ = number of resources reserved for a solitary locust before  <br>
$h$ = foraging rate <br>
$E$ = energy cost for gregarious locusts <br> 
$r$ = resource regeneration rate <br>
Note that $G(t)$ and $S(t)$ actually stay the same, but the energy of locusts' changes

<h3>The model:</h3><br>
<h4>Change in energy for gregarious locusts</h4><br>
$$
\frac{dE_G}{dt} = (\sum_{i=0}^{S(t)}{S(t) \choose i}\lambda^i(1-\lambda)^{S(t)-i}((1-k)\alpha R(t)i))(\frac{S(t)}{S(t)+G(t)})hG(t) + \lambda G(t)(\alpha R(t))(\frac{G(t)}{G(t)+S(t)}) - E
$$
<p>Gregarious locusts gain energy both by discovering resources themselves (second term on the right) and by scrounging off of solitary locusts (first term)</p>


<h4>Change in energy for solitary locusts</h4>
<br>
$$
\frac{dE_S}{dt} = (\sum_{i=0}^{S(t)} {S(t) \choose i} \lambda^i(1-\lambda)^{S(t)-i}k \alpha R(t)i)hS(t)
$$

<h4>Change in resources</h4>
$$
\frac{dR}{dt} = -(\frac{dE_S}{dt}+\frac{dE_G}{dt})+r
$$

In [2]:
#a helper function to compute n choose k
def choose(n,r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom
print(choose(6,2))

15


In [3]:
##implementation of dE_G/dt
def g_dot(S, lamb, R, alpha, k, G, h, E):
    x=0
    for i in range(S):
        x += choose(S,i)*(lamb**i)*(1-lamb)**(S-i)*((1-k)*alpha*R*i)
    x=x*(S/(S+G))*h*G
    x += lamb*G*alpha*R*((G/(S+G)))*h*G - E
    return x

In [4]:
#implementation of dE_S/dt
def s_dot(S, lamb, R, alpha, k, h):
    y = 0
    for i in range(S):
        y += choose(S,i)*((lamb**i)*((1-lamb)**(S-i))*k*alpha*R*i)
    y = y*h*S
    return y

In [5]:
#implementation of dR/dt
def R_dot(x, y, r):
    z = -1*(x+y) + r
    return z

In [11]:
@interact
def simulate(S=(10,50), G=(10,50), lamb=.1, R=30, alpha=.3, k=(.05,1), h=.01, E=.01, r=.2, tsteps=10000):
    dt = .01
    x = np.empty([tsteps + 1,3])
    x[0] = [0,0,R]
    for i in range(tsteps):
        g = g_dot(S, lamb, x[i][2], alpha, k, G, h, E)
        s = s_dot(S, lamb, x[i][2], alpha, k, h)
        R = R_dot(g, s, r)
        x[i+1][0] = x[i][0]+g*dt
        x[i+1][1] = x[i][1]+s*dt
        x[i+1][2] = x[i][2]+R*dt
    g = [y[0] for y in x]
    s = [y[1] for y in x]
    r = [y[2] for y in x]
    plt.plot(range(tsteps+1), g, c='b', label='gregarizing locusts\' energy')
    plt.plot(range(tsteps+1), s, c='r', label='solitary locusts\' energy')
    plt.plot(range(tsteps+1), r, c='g', label='resources')
    legend=plt.legend()
    plt.xlabel('time')
    plt.show()

interactive(children=(IntSlider(value=30, description='S', max=50, min=10), IntSlider(value=30, description='G…

Resources are green, energy of solitary locusts is red, and energy of gregarious locusts is blue